In [3]:
import pandas as pd

In [4]:
path = "../histograms/mlp.hook_out/all_neuron_activations_SD_intervals.csv"

df = pd.read_csv(path)


df.head()

,layer,neuron,sd_-inf_-40_min,sd_-inf_-40_max,activation_-inf_-40_min,activation_-inf_-40_max,count_-inf_-40,sd_-40_-39_min,sd_-40_-39_max,activation_-40_-39_min,...,sd_39_40_min,sd_39_40_max,activation_39_40_min,activation_39_40_max,count_39_40,sd_40_inf_min,sd_40_inf_max,activation_40_inf_min,activation_40_inf_max,count_40_inf
0,blocks.0.hook_mlp_out,0,-inf,-40,NaN,NaN,0,-40,-39,NaN,...,39,40,NaN,NaN,0,40,inf,NaN,NaN,0
1,blocks.0.hook_mlp_out,1,-inf,-40,NaN,NaN,0,-40,-39,NaN,...,39,40,NaN,NaN,0,40,inf,NaN,NaN,0
2,blocks.0.hook_mlp_out,2,-inf,-40,NaN,NaN,0,-40,-39,NaN,...,39,40,NaN,NaN,0,40,inf,NaN,NaN,0
3,blocks.0.hook_mlp_out,3,-inf,-40,NaN,NaN,0,-40,-39,NaN,...,39,40,NaN,NaN,0,40,inf,NaN,NaN,0
4,blocks.0.hook_mlp_out,4,-inf,-40,NaN,NaN,0,-40,-39,NaN,...,39,40,NaN,NaN,0,40,inf,NaN,NaN,0


In [15]:
def get_activation_intervals(df, neuron_idx, layer):
    """
    Retrieves all the non-NaN activation intervals for a given neuron.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the data.
        neuron_idx (int): The index of the neuron.
        layer (str): The name of the layer.

    Returns:
        list: A list of tuples, where each tuple contains the minimum SD value, the minimum activation value, the maximum SD value, and the maximum activation value for a non-NaN activation interval.
    """
    # Find the row with the given neuron index and layer
    row = df[(df['layer'] == layer) & (df['neuron'] == neuron_idx)].iloc[0]

    activation_intervals = []
    for col in df.columns:
        if col.startswith('sd_'):
            sd_range = col.split('_')[1:]
            min_sd = float(sd_range[0])
            max_sd = float(sd_range[-1])
            min_activation_col = f'activation_{col[3:]}_min'
            max_activation_col = f'activation_{col[3:]}_max'
            min_activation = row[min_activation_col]
            max_activation = row[max_activation_col]
            count_col = f'count_{col[3:]}'
            count = row[count_col]
            if count > 0 and not pd.isna(min_activation) and not pd.isna(max_activation):
                activation_intervals.append((min_sd, min_activation, max_sd, max_activation))

    return activation_intervals


intervals = get_activation_intervals(df, neuron_idx=0, layer='blocks.10.hook_mlp_out')
print

[]


In [84]:
import pandas as pd

def get_num_sds_in_interval(df, neuron_idx, layer):
    """
    Retrieves the minimum and maximum standard deviation (SD) values that have non-NaN activation, along with their corresponding activation values, for a given neuron.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the data.
        neuron_idx (int): The index of the neuron.
        layer (str): The name of the layer.

    Returns:
        tuple: A tuple containing the minimum SD value, its corresponding activation value, the maximum SD value, and its corresponding activation value.
    """
    # Find the row with the given neuron index and layer
    row = df[(df['layer'] == layer) & (df['neuron'] == neuron_idx)].iloc[0]

    # Find the minimum and maximum SD values with non-NaN activation
    min_sd, min_activation = float('inf'), None
    max_sd, max_activation = float('-inf'), None
    for col in df.columns:
        if col.startswith('activation_'):
            sd = int(col.split('_')[1])
            value = row[col]
            if not pd.isna(value):
                if sd < min_sd:
                    min_sd = sd
                    min_activation = value
                if sd > max_sd:
                    max_sd = sd
                    max_activation = value

    # Handle the case where all values are NaN
    if min_sd == float('inf') and max_sd == float('-inf'):
        return (None, None, None, None)
    else:
        return (min_sd, min_activation, max_sd, max_activation)
    
# Example usage
num_sds_in_interval = get_num_sds_in_interval(df, neuron_idx=10, layer='blocks.11.hook_mlp_out')
print(f"Number of SDs with values within the interval range: {num_sds_in_interval}")

# Plo

ValueError: invalid literal for int() with base 10: '-inf'

In [82]:
# Ok so the code is

# 1. Get activation interval.

# 2. For each activation interval, get the activation value

1546   NaN
Name: activation_-8_-7_min, dtype: float64

In [11]:
def load_and_visualize(csv_file_path, output_dir):
    # Load the CSV file
    df = pd.read_csv(csv_file_path)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # 1. Heatmap of activation counts across SD intervals
    plt.figure(figsize=(20, 10))
    pivot_df = df.pivot(index='layer', columns='neuron', values='count_-inf_-40')
    for col in df.columns:
        if col.startswith('count_'):
            pivot_df = pivot_df.add(df.pivot(index='layer', columns='neuron', values=col), fill_value=0)
    
    sns.heatmap(pivot_df, cmap='viridis')
    plt.title('Heatmap of Activation Counts Across All SD Intervals')
    plt.xlabel('Neuron')
    plt.ylabel('Layer')
    plt.savefig(os.path.join(output_dir, 'heatmap_activation_counts.png'))
    plt.close()

    # 2. Distribution of activations for extreme SDs
    plt.figure(figsize=(15, 10))
    sns.violinplot(x='layer', y='activation_-inf_-40_min', data=df)
    plt.title('Distribution of Minimum Activations for SD < -40')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'violinplot_min_activations_extreme_negative.png'))
    plt.close()

    plt.figure(figsize=(15, 10))
    sns.violinplot(x='layer', y='activation_40_inf_max', data=df)
    plt.title('Distribution of Maximum Activations for SD > 40')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'violinplot_max_activations_extreme_positive.png'))
    plt.close()

    # 3. Line plot of activation ranges across SD intervals
    sd_ranges = [f"{start}_{end}" for start, end in zip(range(-40, 41), range(-39, 42))]
    
    for layer in df['layer'].unique():
        layer_df = df[df['layer'] == layer]
        plt.figure(figsize=(20, 10))
        
        for i, neuron in enumerate(layer_df['neuron']):
            neuron_data = layer_df[layer_df['neuron'] == neuron]
            min_vals = [neuron_data[f'activation_{r}_min'].values[0] if f'activation_{r}_min' in neuron_data.columns else np.nan for r in sd_ranges]
            max_vals = [neuron_data[f'activation_{r}_max'].values[0] if f'activation_{r}_max' in neuron_data.columns else np.nan for r in sd_ranges]
            
            plt.plot(range(-40, 41), min_vals, alpha=0.1, color='blue')
            plt.plot(range(-40, 41), max_vals, alpha=0.1, color='red')
        
        plt.title(f'Activation Ranges Across SD Intervals - Layer {layer}')
        plt.xlabel('Standard Deviation')
        plt.ylabel('Activation Value')
        plt.savefig(os.path.join(output_dir, f'activation_ranges_layer_{layer}.png'))
        plt.close()

In [12]:
load_and_visualize(path, "histograms/mlp.hook_out/")